In [1]:
import torch, csv, emoji
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sigmoid
import torch.optim as optim
import numpy as np
from transformers import AutoModel, AutoTokenizer 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer

from Bert_NN import read_csv, emojiDataset, toEmoji
from utils import FILEPATH

Using cpu device


In [2]:
a = read_csv(FILEPATH)
print(a[0][-1], a[1][-1])

You people use to pass your boundary on this app. Iya Baba e ni bastard ['👨', '🏃', '📱', '👻']


In [3]:
torch.cuda.is_available()

False

In [4]:
dataset = emojiDataset()

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
data_loader = DataLoader(dataset=dataset, batch_size=16, shuffle=True,num_workers=2)
sentences, emojis, labels_classes = dataset.get_others()

# data_iter = iter(data_loader)
# next_data = data_iter.next()
# features, labels = next_data
# print(features, labels)

toemoji = toEmoji()
learn_rate = optim.Adam(toemoji.parameters(), lr=0.005)
# loss_func = nn.MSELoss()
# loss_func = nn.NLLLoss()
loss_func = nn.BCELoss()
S = Sigmoid()
epochs = 10

In [6]:
for i in range(epochs):
    for data in data_loader:
        features, labels = data
        toemoji.zero_grad()
        results = toemoji(features.float())
        # print(result)
        # print(len(results[0]), len(labels[0]))
        S_results = S(results)
        loss = loss_func(S_results, labels)
        # for result, label in zip(results, labels):
        #     loss += F.nll_loss(result, label) 
        loss.backward()
        learn_rate.step()
    print(loss)

tensor(0.2220, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2040, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.1483, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.1230, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.1695, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2690, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0446, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0436, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0671, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0528, grad_fn=<BinaryCrossEntropyBackward0>)


In [18]:
from collections import defaultdict
# K = 1
collection = defaultdict(int)
predicted = defaultdict(int)
accuracy = 0
# feature, label = dataset[0]

with torch.no_grad():
    for i in range(len(dataset)) :
        feature, label = dataset[i]
        result = toemoji(feature.float())
        # original_str, result_str = '', ''
        # print(result)
        # result = [np.where(elem==emojis[i]) for elem in result if elem.item()>0]
        # print(result)
        # top_k = (-result).argsort()[:K]
        # print(top_k)
        # label_index = np.where(labels_classes==emojis[i])
        # print(label_index)
        # collection[label_index[0][0]] += 1
        # predicted[top_k.item()] += 1

        result = toemoji(feature.float())
        result = [index for index, elem in enumerate(result) if elem.item()>0]
        original_label_index = [index for index, elem in enumerate(label) if elem!=0]
        # print(original_label_index)
        for label_index in result:
            predicted[label_index] += 1
        for label_index in original_label_index:
            collection[label_index] += 1
        
        real_labels = [labels_classes[label_index] for label_index in result]

        # # print(label_index[0][0], top_k.item())
        # if label_index[0][0] == top_k.item(): accuracy+=1
        
        print(f'Sentence: {sentences[i]}\nOriginal labels: {emojis[i]}, output labels: {real_labels}\n') 

        # for top_index in top_k:
        #     result_str += labels_classes[top_index]
        
        # original_str = ''.join(emojis[i])
        
        # print(f'Sentence: {sentences[i]}\nOriginal labels: {original_str}, output labels: {result_str}\n')
for index, label in enumerate(labels_classes):
    print(f'Emoji: {label} has count: {collection[index]} predicted times: {predicted[index]}')
# print(f'Accuracy: {accuracy/len(sentences)}')
        

Sentence: eat then, idiot.
Original labels: ['🍳', '😡'], output labels: ['🍳', '😂']

Sentence: Loving you and having you  is  the most successful achievement in life ♥️✨
Original labels: ['❤', '👨', '👉', '👨', '🎉'], output labels: ['❤', '🎉', '👉']

Sentence: Imagine not being arab 💀💀😭
Original labels: ['🤔', '❌', '👨'], output labels: ['❌']

Sentence: I’m going to a concert tonight and I’m so nervous🤩
Original labels: ['🎵', '🌛', '😍'], output labels: ['🌛', '🎵', '😍']

Sentence: gatekeeping painting…..
Original labels: ['👀', '⛪'], output labels: ['⛪']

Sentence: Not if I see you in water
Original labels: ['❌', '👀', '👨', '💦'], output labels: ['👀', '👨', '💦']

Sentence: yowdi i smell small chismis.
Original labels: ['👨', '🤮'], output labels: ['🤮']

Sentence: A lot of them niggas🤧
Original labels: ['😡'], output labels: ['😡', '🤮']

Sentence: jeez
Original labels: ['🎉'], output labels: []

Sentence: This panther needs Calcium Chloride!
Original labels: ['👨', '💼', '💉'], output labels: ['🍳', '💉', '🤮']

